In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import joblib
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import  WordNetLemmatizer
from nltk.stem import PorterStemmer
stopwords_english = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
csv_files = ["/content/drive/MyDrive/dbeauty.csv","/content/drive/MyDrive/dbeauty2.csv", "/content/drive/MyDrive/dbooks.csv", "/content/drive/MyDrive/delectronics.csv", "/content/drive/MyDrive/dgrocery.csv", "/content/drive/MyDrive/dhome_g.csv"]


In [ ]:
merged_data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

In [ ]:
merged_data.shape

(268753, 2)

In [ ]:
expanded_stop_phrases = [
    "specially designed", "perfect for", "authentically capture", "lighted market style",
    "beautiful stickers", "made from", "ideal for", "great for", "easy to use",
    "high quality", "durable and reliable", "this product is", "excellent choice",
    "long lasting", "premium quality", "perfect gift", "great addition", "versatile",
    "easy storage", "top notch", "state of the art", "precision milled", "beautifully crafted"
]

In [ ]:
# Function to remove stop phrases
def remove_stop_phrases(text, stop_phrases):
    pattern = r'\b(?:' + '|'.join(map(re.escape, stop_phrases)) + r')\b'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002500-\U00002BEF"  # chinese characters
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower().translate(str.maketrans('', '', string.punctuation + string.digits))
    text = remove_emojis(text)  # Remove emojis
    text = remove_stop_phrases(text, expanded_stop_phrases)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\W+', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    word_tokens = word_tokenize(text)

    text_clean = []

    for word in word_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            text_clean.append(word)

    return ' '.join([lemmatizer.lemmatize(word) for word in text_clean])

In [ ]:
merged_data['description'] = merged_data['description'].str.replace(r'\d+', '', regex=True)

In [ ]:
merged_data['description'] = merged_data['description'].apply(clean_text)

In [ ]:
merged_data['description'] = merged_data['description'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

split


In [ ]:
merged_data.shape

(268753, 2)

In [ ]:
merged_data= merged_data.drop_duplicates(subset=['description'])

In [ ]:
merged_data.isnull().sum()

,0
main_category,0
description,0


In [ ]:
merged_data = merged_data.dropna()

In [ ]:
merged_data.shape

(248319, 2)

In [ ]:
merged_data.tail()

,main_category,description
268747,home,classroom american flag standard size approved...
268749,home,planting instruction sowing seed immersed cold...
268750,home,numex big jim listed guinness book world recor...
268751,home,moroccan influence abound geometric design tri...
268752,home,round current outdoor furniture collection cou...


In [ ]:
import os

In [ ]:
merged_data = merged_data.reset_index(drop=True)

In [ ]:
merged_data.isnull().sum()

,0
main_category,0
description,0


In [ ]:
# List of noisy words to remove
noisy_words = {'x', 'inch','mm', 'one', 'use', 'product','new', 'like', 'help', 'would', 'make', 'size', 'first',
               'made', 'work', 'year', 'time', 'water', 'color', 'natural', 'design'}

unwanted_chars = ['Ã¢',]
# Function to remove noisy words from text
def remove_noisy_words(text, noisy_words):
    # Remove specific words using regex
    pattern = r'\b(?:' + '|'.join(map(re.escape, noisy_words)) + r')\b'
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    for char in unwanted_chars:
        cleaned_text = cleaned_text.replace(char, '')
    # Remove extra spaces
    return re.sub(r'\s+', ' ', cleaned_text).strip()

# Apply the function to the 'description' column
merged_data['description'] = merged_data['description'].apply(lambda x: remove_noisy_words(str(x), noisy_words))

Guys we already have the clean_text why do we need this  preprocess_text(text) func? let's remove the repetition

In [ ]:
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    words = text.lower().split()  # Convert to lowercase and split into words
    return [word for word in words if word not in stop_words]  # Remove stopwords

In [ ]:
categories = merged_data['main_category'].unique()
seed_words = {}

for category in categories:
    descriptions = ' '.join(merged_data[merged_data['main_category'] == category]['description'])
    words = preprocess_text(descriptions)
    word_counts = Counter(words)
    seed_words[category] = [word for word, _ in word_counts.most_common(15)]  # Top 5 words

for category, words in seed_words.items():
    print(f"Category: {category}")
    print(f"Seed Words: {words}")
    print()

Category: beauty
Seed Words: ['hair', 'skin', 'nail', 'oil', 'brush', 'dry', 'body', 'wig', 'apply', 'easy', 'material', 'look', 'clean', 'soft', 'eye']

Category: books
Seed Words: ['book', 'author', 'life', 'review', 'story', 'world', 'university', 'read', 'also', 'back', 'way', 'u', 'many', 'well', 'two']

Category: electronics
Seed Words: ['camera', 'usb', 'power', 'case', 'cable', 'feature', 'battery', 'compatible', 'video', 'device', 'screen', 'lens', 'system', 'support', 'card']

Category: grocery
Seed Words: ['flavor', 'tea', 'chocolate', 'taste', 'delicious', 'coffee', 'ingredient', 'organic', 'candy', 'snack', 'sweet', 'food', 'sugar', 'free', 'oz']

Category: home
Seed Words: ['plant', 'garden', 'flag', 'outdoor', 'material', 'easy', 'feature', 'home', 'seed', 'steel', 'flower', 'light', 'durable', 'cover', 'also']



In [ ]:
+merged_data = merged_data.reset_index(drop=True)

In [ ]:
# X = data.drop('main_category', axis=1)
# y = data['main_category']

# Perform stratified train-test split
X_train, X_test= train_test_split(
    merged_data,
    test_size=0.2,  # Adjust the split ratio as needed
    stratify=merged_data['main_category'],  # Ensure balanced categories
    random_state=42  # For reproducibility
)

In [ ]:
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Save the train and test sets
X_train.to_csv(os.path.join(train_dir, 'X_train.csv'), index=False)

X_test.to_csv(os.path.join(test_dir, 'X_test.csv'), index=False)


In [ ]:
merged_data.to_csv("merged_cleaned_data.csv", index=False)